## Overview  
  
The **AzureOpenAI-ResearchPaperAnalyzer** project is designed to facilitate the analysis of research papers using the powerful capabilities of Azure OpenAI. The primary objective of this project is to automate the extraction and categorization of key insights from research papers written in markdown format. By leveraging Azure OpenAI's advanced natural language processing abilities, the script can:  
  
- Process markdown files of research papers.  
- Extract key insights and bullet points from the text.  
- Categorize the extracted information according to different research fields, funding sources, and affiliations.  
- Generate a summary of the paper's content and structure.  
- Format and save the results into a CSV file for easy analysis and reporting.  
  
This tool aims to save researchers significant time and effort by automating the tedious process of reading and summarizing lengthy research papers, allowing them to focus more on the critical aspects of their work.  



In [32]:
from openai import AzureOpenAI

import os
from dotenv import load_dotenv

from IPython.display import Markdown, display, Image

import glob


In [ ]:
load_dotenv()


In [ ]:
# Setting up the deployment name
deployment_name = "gpt-4o-mini"

# The API key for your Azure OpenAI resource.
api_key = os.environ["AZURE_OPENAI_API_KEY"]

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
azure_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']


api_version = "2024-02-15-preview"  # This seems to work

#print the environment variables
print("Azure OpenAI API Key: ", api_key)
print("Azure OpenAI Endpoint: ", azure_endpoint)
print("Azure OpenAI API Version: ", api_version)


client = AzureOpenAI(
  api_key=api_key,  
  azure_endpoint=azure_endpoint,
  api_version=api_version
)

In [ ]:
completion = client.chat.completions.create(
  model=deployment_name,
  messages=[
    {"role": "system", "content": "You are a helpful assistant. Help me with my math homework!"}, # <-- This is the system message that provides context to the model
    {"role": "user", "content": "Hello! Could you solve 2+2?"}  # <-- This is the user message for which the model will generate a response
  ]
)
  
print("Assistant: " + completion.choices[0].message.content)

In [36]:
#load the article file from the markdown folder
article_files = glob.glob("*.md")

#lets limit the files to first 5 for now
article_files = article_files[:5]



In [37]:
#lets chunk the transcript into smaller parts and we want to include the last sentence of the previous chunk in the next chunk to ensure that the context is maintained.

#we will use the split_text function to split the text into smaller parts
def split_text(text, limit):
    """
    Split the text into smaller parts that are less than the limit
    """
    text_parts = []
    current_part = ""
    current_length = 0
    for sentence in text.split("."):
        if current_length + len(sentence) < limit:
            current_part += sentence + "."
            current_length += len(sentence)
        else:
            text_parts.append(current_part)
            current_part = sentence + "."
            current_length = len(sentence)
    text_parts.append(current_part)
    return text_parts


#lets split the full transcript into smaller parts




In [38]:
first_system_message = {"role": "system", "content": """You are an AI assistant specializing in summarizing research papers into concise and meaningful bullet points. You will be provided with chunks of a single research paper, and your task is to analyze each chunk like an experienced researcher. Your goal is to extract and encapsulate the key points in a way that is clear, focused, and free from unnecessary details. Your summary should maintain the academic integrity of the content while being easy to understand.  
  
For each chunk, focus on the following aspects:  
  
1. **Research Focus**:   
   - Identify the primary subject or discipline of the study.   
   - For example: child psychology, sports performance, public health interventions, etc.  
  
2. **Methodology and Tools**:  
   - Specify the scientific methods, techniques, or tools used in the research.  
   - For example: meta-analysis, randomized controlled trials, biomechanical analysis, geospatial modeling, etc.  
  
3. **Keywords and Findings**:  
   - Highlight the main themes, keywords, or notable findings associated with the study.  
   - For example: online parenting programs, health interventions, micro-pacing strategies, etc.  
  
4. **Disciplinary Context**:  
   - Determine the academic field most closely aligned with the study's content, ideally using the ANZSRC classification (if applicable).  
   - For example: psychology, health sciences, engineering, environmental studies, etc.  
  
Your output should be structured as bullet points for clarity. Condense the information into concise statements without sacrificing meaning or relevance.  
  
---  
  
### Example:  
  
#### Input (Chunk of a Research Paper):  
"This study investigates the impact of online parenting programs on improving parenting practices and child outcomes. A meta-analysis was conducted on 25 studies that evaluated the effectiveness of web-based interventions for parents of children aged 3–12 years. Key findings include significant improvements in parenting confidence and reductions in child behavioral problems. The study contributes to the field of child psychology and highlights the importance of accessible digital tools for parenting support."  
  
#### Output (Bullet Points):  
- **Research Focus**: Examines the effectiveness of online parenting programs on parenting practices and child outcomes.  
- **Methodology and Tools**: Meta-analysis of 25 studies on web-based interventions for parents of children aged 3–12 years.  
- **Keywords and Findings**: Parenting confidence, child behavioral problems, digital tools for parenting support.  
- **Disciplinary Context**: Child psychology; aligns with research on digital health interventions in family contexts.  
  
---  
  
### Instructions:  
1. Carefully review each provided chunk of the research paper.  
2. Extract the key points based on the four outlined categories: **Research Focus**, **Methodology and Tools**, **Keywords and Findings**, and **Disciplinary Context**.  
3. Write your output as concise, well-structured bullet points.  
4. Avoid including extraneous details or repetitive information.      
  """}

In [ ]:
#for each article file in the markdown folder call the function to load the data into a variable

for article_file in article_files:
    with open(article_file, 'r', encoding="utf-8") as file:
        article = file.read()
        chunk_size = 8000
        article_length = len(article)
        chunks = article_length//chunk_size
        print(f"The article {article_file} is {article_length} tokens long and will be split into {chunks} chunks.")
        text_parts = split_text(article, chunk_size)
        print(f"Transcript split into {len(text_parts)} parts.")
        #setup the notes.md file using the format file name + notes.md
        notes_file = article_file.replace(".md", "_notes.md")
        with open(notes_file, 'w', encoding='utf-8') as file:
            file.write("# Notes\n\n")
            for i, text_part in enumerate(text_parts):
                print(f"Processing part {i+1}/{len(text_parts)}")
                completion = client.chat.completions.create(
                    model=deployment_name,
                    messages=[first_system_message, {"role": "user", "content": text_part}]
                )
                response = completion.choices[0].message.content
                file.write(f"## Part {i+1}\n\n")
                file.write(f"{response}\n\n")
                #print(response)
        
  

In [ ]:
#lets run three system prompts across each of the _notes.md files to get the final output

#load the article file from the markdown folder
article_notes = glob.glob("*_notes.md")

#lets limit the files to first 2 for now
article_notes = article_notes[:5]

article_notes

In [54]:
field_of_research_code_system_prompt = {"role": "system", "content": """You are an expert in the **2020 Australian and New Zealand Standard Research Classification (ANZSRC)** system, specifically the **Fields of Research (FoR)** codes. Your task is to analyze the notes or description of a research paper and identify the most appropriate **4-digit Fields of Research (FoR) code** that corresponds to the subject matter of the research.    
  
The ANZSRC FoR classification system is hierarchical:    
1. **2-digit codes** represent broad research divisions (e.g., 42 Health Sciences).    
2. **4-digit codes** represent specific research groups within those divisions (e.g., 4206 Public Health).    
3. **6-digit codes** provide even finer granularity but are not required for this task.    
  
## Your Task:  
1. Focus on the **4-digit FoR code** level to classify the research into its most relevant research group.    
2. Analyze all notes provided about the study, including its focus, methodology, subject matter, keywords, and implications, to determine the **primary field of research**.    
3. Prioritize the **primary discipline** of the study rather than secondary or interdisciplinary areas, even if the study spans multiple fields.    
4. Where relevant, you may also suggest **secondary FoR codes** that reflect interdisciplinary aspects of the research.  
  
---  
  
## Guidance for Identifying the Correct 4-Digit FoR Code:  
- **Research Focus**: What is the primary subject or discipline of the study? (e.g., child psychology, sports performance, public health interventions).    
- **Methodology and Tools**: What scientific methods, techniques, or tools were used? (e.g., meta-analysis, randomized controlled trials, biomechanical analysis).    
- **Keywords and Findings**: What are the main themes or keywords associated with the research? (e.g., online parenting programs, health interventions, micro-pacing strategies).    
- **Disciplinary Context**: Which academic field does the study's content align with most closely within the ANZSRC classification?    
  
If the research overlaps multiple disciplines, choose the **primary FoR code** that best represents the core focus of the study. Use the content, keywords, and context provided in the notes to guide your classification.  
  
---  
  
## Examples:  
  
### Example 1:  
**Input**: "This research examines the biomechanics of elite swimmers to improve stroke efficiency."    
**Output**: "4207 – Sports Science and Exercise"    
  
### Example 2:  
**Input**: "This study investigates the impact of social media algorithms on user behavior and engagement."    
**Output**: "5205 – Communication and Media Studies"    
  
### Example 3:  
**Input**: "This paper analyzes the genetic basis of drought resistance in wheat using genome-wide association studies."    
**Output**: "3105 – Genetics"    
  
---  
  
### Analyze the following notes and provide the **4-digit Fields of Research (FoR) code**:  
""" } # Update prompt for FOR Code detection here if needed"}

In [1]:
funding_sources_system_prompt = {"role": "system", "content": """You are an expert research assistant tasked with identifying funding sources for academic studies. Your job is to analyze detailed notes about a research paper, which may be divided into multiple parts but pertains to a single study. Your goal is to extract all mentions of funding sources or statements indicating the lack of funding from the notes.  
  
Carefully review all parts of the notes as a single cohesive input and provide one of the following outputs:  
1. A list of funding sources mentioned in the notes, including grant names, funding agencies, organizations, or any other details related to funding.  
2. If no funding information is mentioned in the notes, explicitly state: "No funding sources were reported in the provided notes."  
  
### Examples:  
Input:  
"The research was funded by the European Research Council (ERC Starting Grant 6789). Additional funding was provided by the Swedish Research Council."  
Output:  
"Funding Sources: European Research Council (ERC Starting Grant 6789), Swedish Research Council."  
  
Input:  
"The authors declared no funding sources for this research. Ethical approval was obtained from the university ethics committee."  
Output:  
"No funding sources were reported in the provided notes."  
  
Input:  
"The study acknowledges financial support from the National Institutes of Health (NIH grant R01-AB123456)."  
Output:  
"Funding Sources: National Institutes of Health (NIH grant R01-AB123456)."  
  
Now, analyze the following notes and extract all funding-related information:  ."""}

In [ ]:
university = "La Trobe University"

In [ ]:
content = f"""You are an expert research assistant tasked with identifying affiliations to a specific university in academic research. Your job is to analyze detailed notes about a research paper, which may be divided into multiple parts but pertains to a single study. The university name will be provided as a placeholder `{university}`, and you must check if any authors or affiliations mentioned in the notes are explicitly connected to this university.  
  
Carefully review all parts of the notes as a single cohesive input and provide one of the following outputs:  
1. If any author or affiliation is explicitly linked to `{university}`, list the relevant details (e.g., author name, department, or any specific mention of the university).  
2. If no affiliation to `{university}` is mentioned in the notes, explicitly state: "No affiliations to {university} were reported in the provided notes."  
  
### Guidelines:  
- Look for explicit mentions of `{university}` in any part of the notes, including author affiliations, acknowledgments, or other sections.  
- Treat the notes as a single cohesive input, even if they are divided into multiple parts.  
- Only report affiliations explicitly linked to `{university}`. Do not infer or assume affiliations based on other universities or organizations.  
  
### Examples:  
Input:  
"University: La Trobe University    
Notes:    
- Authors: Dr. John Smith (La Trobe University, Department of Sports Science), Dr. Jane Doe (University of Melbourne).    
- Acknowledgments: The authors thank La Trobe University for providing access to research facilities."  
Output:  
"Affiliations to La Trobe University: Dr. John Smith (Department of Sports Science), acknowledgment of research facilities."  
  
Input:  
"university: La Trobe University    
Notes:    
- Authors: Dr. Emily Brown (University of Sydney), Dr. Mark Wilson (Monash University).    
- No mention of La Trobe University in the acknowledgments or affiliations."  
Output:  
"No affiliations to La Trobe University were reported in the provided notes."  
  
Input:  
"university: University of Sydney    
Notes:    
- Authors: Dr. Alex Green (University of Sydney, Faculty of Medicine), Dr. Lisa White (University of Queensland).    
- The study was supported by the University of Sydney research grant."  
Output:  
"Affiliations to University of Sydney: Dr. Alex Green (Faculty of Medicine), research grant support."""



In [58]:
affiliation_system_prompt = {"role": "system", "content": content}


In [59]:
md_friendly_format_system_prompt = {"role": "system", "content": """YYou are an expert research assistant tasked with consolidating the results of multiple analyses into a structured Markdown table. Each iteration will provide the following information about a research paper:  
1. **Title of the Paper:** The title of the research paper being analyzed.  
2. **FoR Code Classification:** The primary Fields of Research (FoR) 4-digit code that best represents the study, along with the reasons for selecting this code.  
3. **Funding Sources Extraction:** A list of funding sources mentioned in the notes or confirmation that no funding was reported.  
4. **University Affiliations Extraction:** A list of affiliations to a specific university or confirmation that no affiliations were reported.  
  
Your task is to:  
1. Extract the most important pieces of information from the results.  
2. Combine this information into **a single Markdown table row**, excluding the header row and divider row, as they are already pre-generated.  
  
### Output Format:  
The output must be a **Markdown-formatted row** for the table, with data separated by vertical bars (`|`). Do not include the header or divider rows. Make sure the row has the following structure:  
| [Title] | [FoR Code] | [Reason for FoR Code] | [Funding Sources] | [Affiliations] |   """}


In [ ]:
#setup a csv file with the following columns Article, Field of Research Code, Reasoning, Funding Sources, Affiliations
#this file will be appeneded with the results of the system prompts for each article

#setup the output markdown file
with open("results.md", 'w', encoding='utf-8') as file:  
    file.write("""\  
| Title                                   | FoR Code | Reason for FoR Code                     | Funding Sources          | Affiliations to University           |  
|-----------------------------------------|----------|-----------------------------------------|--------------------------|---------------------------------------|  
""")
#lets run the system prompts across each of the _notes.md files to get the final output


for article_note in article_notes:
    with open(article_note, 'r', encoding="utf-8") as file:
        article = file.read()
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[field_of_research_code_system_prompt, {"role": "user", "content": article}]
        )
        fieldOfResearchResult = completion.choices[0].message.content
        print(fieldOfResearchResult)

        completion = client.chat.completions.create(
            model=deployment_name,
            messages=[funding_sources_system_prompt, {"role": "user", "content": article}]
        )
        fundingSourcesResult = completion.choices[0].message.content
        print(fundingSourcesResult)

        completion = client.chat.completions.create(
            model=deployment_name,
            messages=[affiliation_system_prompt, {"role": "user", "content": article}]
        )  
        affiliationsResult = completion.choices[0].message.content
        print(affiliationsResult)

        #lets collate the results into a string
        collatedResults = f"{fieldOfResearchResult}, {fundingSourcesResult}, {affiliationsResult}"

        #lets call the completion function to get a csv friendly response for the results
        completion = client.chat.completions.create(
            model=deployment_name,
            messages=[md_friendly_format_system_prompt, {"role": "user", "content": collatedResults}]
        )
        md_friendly_format = completion.choices[0].message.content
        #append to the markdown file results.md with the results
        with open("results.md", 'a', encoding='utf-8') as file:
            file.write(f"{md_friendly_format}\n")
        print(md_friendly_format)
        print("\n\n")
        
        # #print the response
        # print(f"Article: {article_file}")
        # print(f"Field of Research Code: {fieldOfResearchResult}")
        # print(f"Funding Sources: {fundingSourcesResult}")
        # print(f"Affiliations: {affiliationsResult}")
        # #print a new line
        # print("\n\n")

